In [1]:
import os
import re


In [2]:
# Load the stdio file into a string
with open('stdio', 'r') as f:
    stdio = f.read()


In [61]:
# ignore the first section where only the tests executions are printed
# look for the last "In execution:" and then cut everything before it

last_in_execution = stdio.rfind('In execution:')
stdio=stdio[last_in_execution:]

In [3]:
# # we need to find valgrind supression outputs
# # Its a block of text starts with:
# # ==2241829==
# #{
# # and ends with
# #}
# #==2241829== LEAK SUMMARY:
# # where 2241829 is any number

# match_group_pattern = r"==\d+==\n(\{.*?\}\n)==\d+== LEAK SUMMARY:"

# list_of_matches = re.findall(match_group_pattern, stdio, re.DOTALL)


In [4]:
# len(list_of_matches)

0

In [63]:
# == \n{\
match_top = r"==\d+== \n\{"
# \n}\n==1756825==
# match_bottom = r"==\d+== LEAK SUMMARY:"
match_bottom = r"\n\}\n==\d+=="

match_top_str = re.findall(match_top, stdio)
match_bottom_str = re.findall(match_bottom, stdio)
bottom_index = []
top_index = []
for bottom in match_bottom_str:
    bottom_index.append(stdio.find(bottom)+2)

for top in match_top_str:
    # append the index of the last character of the match

    top_index.append(stdio.find(top)+len(top)-1)

In [64]:
len(top_index), len(bottom_index)

(528, 532)

In [65]:
knonw_sup_hash = []
for i in range(len(top_index)):
    start=top_index[i]
    end = bottom_index[i]
    suppression = stdio[start:end]
    # print(suppression)
    # if the suppression is not unique, append this to a file suppressions.txt
    sup_hash = hash(suppression)
    if sup_hash not in knonw_sup_hash:
        # write to file
        with open('suppressions.txt', 'a') as f:
            f.write(suppression)
        knonw_sup_hash.append(sup_hash)
    else:
        pass



In [40]:
loc = bottom_index[0]
(stdio[loc-300:loc+100])

'u/ld-2.31.so)\n==1756825== \n{\n   <insert_a_suppression_name_here>\n   Memcheck:Leak\n   match-leak-kinds: reachable\n   fun:malloc\n   fun:gomp_malloc\n   fun:gomp_init_num_threads\n   fun:initialize_env\n   fun:call_init.part.0\n   fun:call_init\n   fun:_dl_init\n   obj:/usr/lib/x86_64-linux-gnu/ld-2.31.so\n}\n==1756825== LEAK SUMMARY:\n==1756825==    definitely lost: 0 bytes in 0 blocks\n==1756825==    indirec'